# Original for testing perposes

In [ ]:

import random
from collections import defaultdict

def generate_random_graph(num_nodes, num_edges):
    graph = {chr(65 + i): [] for i in range(num_nodes)}
    if num_edges > num_nodes * (num_nodes - 1) // 2:
        print("Warning: Number of edges exceeds maximum possible edges for this number of nodes.")
        num_edges = num_nodes * (num_nodes - 1) // 2

    edges = set()
    keys = list(graph.keys())
    while len(edges) < num_edges:
        u = random.choice(keys)
        v = random.choice(keys)
        if u != v and (u, v) not in edges and (v, u) not in edges:
            edges.add((u, v))
            graph[u].append(v)
            graph[v].append(u)

    return graph
def find_left_end(curr, CombForest):
    # ascend left to the leftmost node (stop when parentleft is None)
    while CombForest[curr]['parentleft'] is not None:
        curr = CombForest[curr]['parentleft']
    return curr

def find_right_end(curr, CombForest):
    while CombForest[curr]['parentright'] is not None:
        curr = CombForest[curr]['parentright']
    return curr

# --- Example random graph ---
num_nodes = 10
num_edges = 40
random_graph = generate_random_graph(num_nodes, num_edges)

print(f"Random graph with {num_nodes} nodes and {num_edges} edges:")
for node, neighbors in random_graph.items():
    print(f"{node}: {neighbors}")

# --- Comb forest construction (no extra front nodes, only u-v keys) ---
processed_edges = set()
CombForest = {}
last_attach = {}   # current attach point for each original node u
prev_u = None      # oldU

# initialize CombForest for original nodes
for n in random_graph.keys():
    CombForest[n] = {'parentleft': None, 'parentright': None, 'children': []}
    last_attach[n] = n

# process each undirected edge only once
for node, neighbors in random_graph.items():
    u = node
    for neighbor in neighbors:
        edge_key = f"{node}-{neighbor}" if node < neighbor else f"{neighbor}-{node}"
        if edge_key in processed_edges:
            continue
        processed_edges.add(edge_key)

        # create the edge node in the CombForest
        CombForest[edge_key] = {'parentleft': None, 'parentright': None, 'children': []}

        v = neighbor

        # ensure v exists (should), then find its leftmost attach
        if v not in CombForest:
            CombForest[v] = {'parentleft': None, 'parentright': None, 'children': []}
            last_attach[v] = v
        attach_v = find_left_end(v, CombForest)  # always ascend left for v

        # get current attach for u (the front)
        if u not in CombForest:
            CombForest[u] = {'parentleft': None, 'parentright': None, 'children': []}
            last_attach[u] = u
        current_attach_u = last_attach[u]

        # If previous u is different, "go left" on u's front by making this edge the parentleft
        print("prev_u",prev_u)
        print("u",u)
        if prev_u != u:
            print(" i got here")
            # make the new edge the left parent of current_attach_u
            current_attach_u = find_left_end(u, CombForest)
            print("current_attach_u",current_attach_u,"=",edge_key)

            CombForest[current_attach_u]['parentleft'] = edge_key
            CombForest[edge_key]['children'].append(current_attach_u)
        else:
            # prev_u is same as u (or None) -> attach the edge to the attach point
            if current_attach_u == u:
                CombForest[current_attach_u]['parentleft'] = edge_key
            else:
                CombForest[current_attach_u]['parentright'] = edge_key
            CombForest[edge_key]['children'].append(current_attach_u)

        # Attach the v-side (always use left ascend result attach_v)

        attach_v = find_left_end(v, CombForest)
        CombForest[attach_v]['parentleft'] = edge_key
        CombForest[edge_key]['children'].append(attach_v)

        # After attaching, the edge becomes the new attach point on u's front
        last_attach[u] = edge_key

        # update prev_u
        prev_u = u

def compute_max_depth(CombForest):
    # Helper to compute max ascent depth from all original nodes (measure of imbalance)
    def ascent_depth(node):
        if node not in CombForest:
            return 0
        depth_left = 1 + ascent_depth(CombForest[node]['parentleft']) if CombForest[node]['parentleft'] else 0
        depth_right = 1 + ascent_depth(CombForest[node]['parentright']) if CombForest[node]['parentright'] else 0
        return max(depth_left, depth_right)

    original_nodes = [k for k in CombForest if '-' not in k]
    return max(ascent_depth(node) for node in original_nodes)


# finished, print summary
print("\nCombForest (summary):")
for k, v in CombForest.items():
    print(f"{k}: {v}")
max_depth = compute_max_depth(CombForest)
print(f"Max ascent depth Original: {max_depth}")

Random graph with 10 nodes and 40 edges:
A: ['C', 'E', 'I', 'G', 'D', 'J', 'B', 'F']
B: ['C', 'G', 'H', 'J', 'A', 'I', 'F']
C: ['A', 'B', 'H', 'I', 'G', 'F', 'E', 'D']
D: ['J', 'E', 'H', 'A', 'I', 'F', 'G', 'C']
E: ['D', 'A', 'I', 'G', 'J', 'C', 'F', 'H']
F: ['G', 'I', 'C', 'E', 'D', 'A', 'B', 'J']
G: ['B', 'F', 'H', 'A', 'E', 'C', 'J', 'I', 'D']
H: ['C', 'D', 'B', 'G', 'J', 'I', 'E']
I: ['C', 'A', 'E', 'F', 'G', 'D', 'J', 'H', 'B']
J: ['D', 'B', 'G', 'A', 'E', 'H', 'I', 'F']
prev_u None
u A
 i got here
current_attach_u A = A-C
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u B
 i got here
current_attach_u A-B = B-C
prev_u B
u B
prev_u B
u B
prev_u B
u B
prev_u B
u B
prev_u B
u B
prev_u B
u C
 i got here
current_attach_u B-C = C-H
prev_u C
u C
prev_u C
u C
prev_u C
u C
prev_u C
u C
prev_u C
u C
prev_u C
u D
 i got here
current_attach_u C-D = D-J
prev_u D
u D
prev_u D
u D
prev_u D
u D
prev_u D
u D
prev_u D
u D
prev_u D
u E
 i got here

# Adding orthogonal codes

In [ ]:
import numpy as np
import random

def generate_random_graph(num_nodes, num_edges):
    graph = {chr(65 + i): [] for i in range(num_nodes)}
    if num_edges > num_nodes * (num_nodes - 1) // 2:
        print("Warning: Number of edges exceeds maximum possible edges for this number of nodes.")
        num_edges = num_nodes * (num_nodes - 1) // 2

    edges = set()
    keys = list(graph.keys())
    while len(edges) < num_edges:
        u = random.choice(keys)
        v = random.choice(keys)
        if u != v and (u, v) not in edges and (v, u) not in edges:
            edges.add((u, v))
            graph[u].append(v)
            graph[v].append(u)

    return graph

def assign_codes(graph):
    # Extract unique edges as sorted tuples
    edges = set()
    for u in graph:
        for v in graph[u]:
            edge = (min(u, v), max(u, v))
            edges.add(edge)
    edge_list = list(edges)
    num_edges = len(edge_list)

    # Assign codes
    codes = {}
    for node in graph:
        vec = np.zeros(num_edges)
        for i, edge in enumerate(edge_list):
            if node in edge:
                vec[i] = 1
        codes[node] = vec

    return codes
import numpy as np

def find_neighbors(codes):
    """
    Given a dictionary of node codes (numpy arrays), find each node's neighbors
    by computing dot products. Neighbors are nodes where the dot product is non-zero.
    """
    neighbors = {}
    nodes = list(codes.keys())
    for i, node in enumerate(nodes):
        neighbors[node] = []
        for other in nodes:
            if node != other and np.dot(codes[node], codes[other]) != 0:
                neighbors[node].append(other)
    return neighbors
graph=generate_random_graph(5, 10)
print(graph)
graph=assign_codes(graph)
print(graph)
nb=find_neighbors(graph)
print(nb)

{'A': ['B', 'E', 'C', 'D'], 'B': ['D', 'E', 'A', 'C'], 'C': ['A', 'D', 'B', 'E'], 'D': ['B', 'E', 'A', 'C'], 'E': ['B', 'A', 'D', 'C']}
{'A': array([1., 1., 1., 0., 0., 0., 0., 0., 0., 1.]), 'B': array([0., 0., 1., 1., 1., 0., 1., 0., 0., 0.]), 'C': array([0., 1., 0., 0., 1., 0., 0., 1., 1., 0.]), 'D': array([1., 0., 0., 0., 0., 1., 1., 1., 0., 0.]), 'E': array([0., 0., 0., 1., 0., 1., 0., 0., 1., 1.])}
{'A': ['B', 'C', 'D', 'E'], 'B': ['A', 'C', 'D', 'E'], 'C': ['A', 'B', 'D', 'E'], 'D': ['A', 'B', 'C', 'E'], 'E': ['A', 'B', 'C', 'D']}


# Building comb forest with  Orthogonal codes

In [ ]:
import random
from collections import defaultdict
import numpy as np

def generate_random_graph(num_nodes, num_edges):
    graph = {chr(65 + i): [] for i in range(num_nodes)}
    if num_edges > num_nodes * (num_nodes - 1) // 2:
        print("Warning: Number of edges exceeds maximum possible edges for this number of nodes.")
        num_edges = num_nodes * (num_nodes - 1) // 2

    edges = set()
    keys = list(graph.keys())
    while len(edges) < num_edges:
        u = random.choice(keys)
        v = random.choice(keys)
        if u != v and (u, v) not in edges and (v, u) not in edges:
            edges.add((u, v))
            graph[u].append(v)
            graph[v].append(u)

    return graph

def find_left_end(curr, CombForest):
    # ascend left to the leftmost node (stop when parentleft is None)
    while CombForest[curr]['parentleft'] is not None:
        curr = CombForest[curr]['parentleft']
    return curr

def find_right_end(curr, CombForest):
    while CombForest[curr]['parentright'] is not None:
        curr = CombForest[curr]['parentright']
    return curr

# --- Comb forest construction (modified to assign codes) ---
def build_comb_forest(random_graph):
    processed_edges = set()
    CombForest = {}
    last_attach = {}  # current attach point for each original node u
    prev_u = None  # oldU

    # initialize CombForest for original nodes
    for n in random_graph.keys():
        CombForest[n] = {'parentleft': None, 'parentright': None, 'children': [], 'code': None}  # code to be assigned later
        last_attach[n] = n

    # process each undirected edge only once
    for node, neighbors in random_graph.items():
        u = node
        for neighbor in neighbors:
            edge_key = f"{node}-{neighbor}" if node < neighbor else f"{neighbor}-{node}"
            if edge_key in processed_edges:
                continue
            processed_edges.add(edge_key)

            # create the edge node in the CombForest
            CombForest[edge_key] = {'parentleft': None, 'parentright': None, 'children': [], 'code': None}  # code later

            v = neighbor
            # ensure v exists (should), then find its leftmost attach
            if v not in CombForest:
                CombForest[v] = {'parentleft': None, 'parentright': None, 'children': [], 'code': None}
                last_attach[v] = v
            attach_v = find_left_end(v, CombForest)  # always ascend left for v

            # get current attach for u (the front)
            if u not in CombForest:
                CombForest[u] = {'parentleft': None, 'parentright': None, 'children': [], 'code': None}
                last_attach[u] = u
            current_attach_u = last_attach[u]

            # If previous u is different, "go left" on u's front by making this edge the parentleft
            if prev_u != u:
                # make the new edge the left parent of current_attach_u
                current_attach_u = find_left_end(u, CombForest)
                CombForest[current_attach_u]['parentleft'] = edge_key
                CombForest[edge_key]['children'].append(current_attach_u)
            else:
                # prev_u is same as u (or None) -> attach the edge to the attach point
                if current_attach_u == u:
                    CombForest[current_attach_u]['parentleft'] = edge_key
                else:
                    CombForest[current_attach_u]['parentright'] = edge_key
                CombForest[edge_key]['children'].append(current_attach_u)

            # Attach the v-side (always use left ascend result attach_v)
            attach_v = find_left_end(v, CombForest)
            CombForest[attach_v]['parentleft'] = edge_key
            CombForest[edge_key]['children'].append(attach_v)

            # After attaching, the edge becomes the new attach point on u's front
            last_attach[u] = edge_key

            # update prev_u
            prev_u = u

    # Now assign codes
    # List all edge keys and assign indices
    edge_list = list(processed_edges)
    num_edges = len(edge_list)
    edge_index = {edge: idx for idx, edge in enumerate(edge_list)}

    # For original nodes: vector with 1s on incident edges
    for node in random_graph:
        vec = np.zeros(num_edges)
        for neighbor in random_graph[node]:
            edge_key = f"{node}-{neighbor}" if node < neighbor else f"{neighbor}-{node}"
            idx = edge_index[edge_key]
            vec[idx] = 1
        CombForest[node]['code'] = vec

    # For edge nodes: vector with 1 only on its own index
    for edge_key in edge_list:
        vec = np.zeros(num_edges)
        idx = edge_index[edge_key]
        vec[idx] = 1
        CombForest[edge_key]['code'] = vec

    return CombForest

random_graph = generate_random_graph(5, 10)
CombForest=build_comb_forest(random_graph)
# finished, print summary
print("\nOriginal Graph :")
for k, v in random_graph.items():
    print(f"{k}: {v}")
print("\nCombForest :")
for k, v in CombForest.items():
    print(f"{k}: {v}")



Original Graph :
A: ['D', 'C', 'E', 'B']
B: ['C', 'E', 'D', 'A']
C: ['D', 'B', 'A', 'E']
D: ['A', 'C', 'E', 'B']
E: ['B', 'D', 'A', 'C']

CombForest :
A: {'parentleft': 'A-D', 'parentright': None, 'children': [], 'code': array([0., 0., 1., 0., 0., 0., 1., 1., 0., 1.])}
B: {'parentleft': 'A-B', 'parentright': None, 'children': [], 'code': array([0., 0., 0., 1., 1., 0., 0., 1., 1., 0.])}
C: {'parentleft': 'A-C', 'parentright': None, 'children': [], 'code': array([0., 1., 0., 0., 0., 1., 1., 0., 1., 0.])}
D: {'parentleft': 'A-D', 'parentright': None, 'children': [], 'code': array([1., 0., 0., 0., 1., 1., 0., 0., 0., 1.])}
E: {'parentleft': 'A-E', 'parentright': None, 'children': [], 'code': array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0.])}
A-D: {'parentleft': 'B-D', 'parentright': 'A-C', 'children': ['A', 'D'], 'code': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])}
A-C: {'parentleft': 'B-C', 'parentright': 'A-E', 'children': ['A-D', 'C'], 'code': array([0., 0., 0., 0., 0., 0., 1., 0., 0

# previous Version left for when needed

In [ ]:
from collections import defaultdict, deque
import numpy as np

class ActivationNeighborFinder:
    def __init__(self, CombForest):
        self.CombForest = CombForest

    # Multi-layer Activation with Inhibition by Orthogonal Codes
    def find_neighbors_inhibition_activation(self, u):
        """
        Nodes can be activated or inhibited based on code orthogonality.
        A node is inhibited if its code is orthogonal (dot product == 0) to the starting node's code.
        Inhibited nodes block signal propagation. Creates more selective neighbor discovery.
        """
        activation_state = defaultdict(lambda: 'neutral')  # 'active', 'inhibited', 'neutral'
        neighbors = set()
        u_code = self.CombForest[u]['code']

        # Start with target node active
        activation_state[u] = 'active'

        # Inhibit nodes that are orthogonal to u
        all_nodes = list(self.CombForest.keys())
        for node in all_nodes:
            if node != u:
                node_code = self.CombForest[node]['code']
                if np.dot(u_code, node_code) == 0:
                    activation_state[node] = 'inhibited'

        queue = deque([u])
        visited = set()

        while queue:
            curr = queue.popleft()
            if curr in visited or activation_state[curr] == 'inhibited':
                continue
            visited.add(curr)

            # Activate current node
            activation_state[curr] = 'active'

            if '-' in curr:
                x, y = curr.split('-')
                if u == x:
                    neighbors.add(y)
                elif u == y:
                    neighbors.add(x)

            # Propagate only if not inhibited
            for parent_key in ['parentleft', 'parentright']:
                parent = self.CombForest[curr].get(parent_key)
                if (parent and parent not in visited and
                    activation_state[parent] != 'inhibited'):
                    queue.append(parent)

        return neighbors, dict(activation_state)

    # Method 6: Mutual Neighbors with Cross-Activation
    def find_mutual_neighbors_cross_activation(self, u, v, sync_threshold=1):
        """
        Find mutual neighbors by having two activation processes (from u and v)
        that must synchronize at edge nodes to reveal mutual connections.
        """
        # Run dual activation processes
        activation_u = defaultdict(float)
        activation_v = defaultdict(float)
        neighbors_u = defaultdict(float)
        neighbors_v = defaultdict(float)
        mutual_neighbors = set()

        # Initialize
        activation_u[u] = 1.0
        activation_v[v] = 1.0

        # Simultaneous BFS from both nodes
        queue_u = deque([(u, 1.0)])
        queue_v = deque([(v, 1.0)])
        visited_u = set()
        visited_v = set()

        max_steps = 10
        step = 0

        while (queue_u or queue_v) and step < max_steps:
            step += 1

            # Process u's activation
            if queue_u:
                curr_u, energy_u = queue_u.popleft()
                if curr_u not in visited_u and energy_u > 0.1:
                    visited_u.add(curr_u)
                    activation_u[curr_u] = energy_u

                    if '-' in curr_u:
                        x, y = curr_u.split('-')
                        if x == u:
                            neighbors_u[y] = max(neighbors_u[y], energy_u)
                        elif y == u:
                            neighbors_u[x] = max(neighbors_u[x], energy_u)

                    for parent_key in ['parentleft', 'parentright']:
                        parent = self.CombForest[curr_u].get(parent_key)
                        if parent and parent not in visited_u:
                            queue_u.append((parent, energy_u * 0.8))

            # Process v's activation
            if queue_v:
                curr_v, energy_v = queue_v.popleft()
                if curr_v not in visited_v and energy_v > 0.1:
                    visited_v.add(curr_v)
                    activation_v[curr_v] = energy_v

                    if '-' in curr_v:
                        x, y = curr_v.split('-')
                        if x == v:
                            neighbors_v[y] = max(neighbors_v[y], energy_v)
                        elif y == v:
                            neighbors_v[x] = max(neighbors_v[x], energy_v)

                    for parent_key in ['parentleft', 'parentright']:
                        parent = self.CombForest[curr_v].get(parent_key)
                        if parent and parent not in visited_v:
                            queue_v.append((parent, energy_v * 0.8))

        # Find mutual neighbors where strengths synchronize
        for w in set(neighbors_u.keys()) & set(neighbors_v.keys()):
            if w != u and w != v:
                combined_strength = neighbors_u[w] + neighbors_v[w]
                if combined_strength > sync_threshold:
                    mutual_neighbors.add(w)

        return mutual_neighbors, (activation_u, activation_v)


# Example usage functions
def demonstrate_activation_methods(CombForest, test_node):
    """Demonstrate all activation methods"""
    finder = ActivationNeighborFinder(CombForest)

    print(f"\n=== Activation Methods for Node '{test_node}' ===")

    # Method 5: Inhibition activation
    neighbors5, states = finder.find_neighbors_inhibition_activation(test_node)
    print(f"5. Inhibition Activation: {neighbors5}")

    # Compare with original method
    original_neighbors = find_neighbors_original(test_node, CombForest)
    print(f"Original Method: {original_neighbors}")

    return {
        'inhibition': neighbors5,
        'original': original_neighbors
    }

def find_neighbors_original(u, CombForest):
    """Original neighbor finding method for comparison"""
    neighbors = set()
    from collections import deque
    queue = deque([u])
    visited = set()
    while queue:
        curr = queue.popleft()
        if curr in visited:
            continue
        visited.add(curr)
        pl = CombForest[curr]['parentleft']
        if pl is not None and pl not in visited:
            queue.append(pl)
        pr = CombForest[curr]['parentright']
        if pr is not None and pr not in visited:
            queue.append(pr)
        if '-' in curr:
            x, y = curr.split('-')
            if u == x:
                neighbors.add(y)
            elif u == y:
                neighbors.add(x)
    return neighbors
Results=demonstrate_activation_methods(CombForest, 'A')

Resul=demonstrate_activation_methods(CombForest, 'B')
print("-------- Mutual--------------")
finder = ActivationNeighborFinder(CombForest)
Mn=finder.find_mutual_neighbors_cross_activation('A', 'B')
print(Mn)


=== Activation Methods for Node 'A' ===
5. Inhibition Activation: {'D', 'C', 'B', 'E'}
Original Method: {'D', 'C', 'B', 'E'}

=== Activation Methods for Node 'B' ===
5. Inhibition Activation: {'A', 'C', 'D', 'E'}
Original Method: {'A', 'C', 'D', 'E'}
-------- Mutual--------------
({'D', 'C', 'E'}, (defaultdict(<class 'float'>, {'A': 1.0, 'A-D': 0.8, 'B-D': 0.6400000000000001, 'A-C': 0.6400000000000001, 'C-D': 0.5120000000000001, 'B-C': 0.5120000000000001, 'A-E': 0.5120000000000001, 'D-E': 0.40960000000000013, 'C-E': 0.40960000000000013, 'B-E': 0.40960000000000013}), defaultdict(<class 'float'>, {'B': 1.0, 'A-B': 0.8, 'B-C': 0.6400000000000001, 'C-D': 0.5120000000000001, 'B-E': 0.5120000000000001, 'D-E': 0.40960000000000013, 'C-E': 0.40960000000000013, 'B-D': 0.40960000000000013})))


# Activation

In [ ]:
from collections import defaultdict, deque
import numpy as np

class ActivationNeighborFinder:
    def __init__(self, CombForest):
        self.CombForest = CombForest

    def get_connected(self, node, target_code):
        if '-' not in node:
            if np.dot(self.CombForest[node]['code'], target_code) != 0:
                return [node]
            else:
                return []
        else:
            connected = []
            for child in self.CombForest[node]['children']:
                connected += self.get_connected(child, target_code)
            return connected

    def find_neighbors_inhibition_activation(self, u):
        """
        Nodes can be activated or inhibited based on code orthogonality.
        A node is inhibited if its code is orthogonal (dot product == 0) to the starting node's code.
        Inhibited nodes block signal propagation. Creates more selective neighbor discovery.
        """
        activation_state = defaultdict(lambda: 'neutral')  # 'active', 'inhibited', 'neutral'
        neighbors = {}
        u_code = self.CombForest[u]['code']

        # Start with target node active
        activation_state[u] = 'active'

        # Inhibit nodes that are orthogonal to u
        all_nodes = list(self.CombForest.keys())
        for node in all_nodes:
            if node != u:
                node_code = self.CombForest[node]['code']
                if np.dot(u_code, node_code) == 0:
                    activation_state[node] = 'inhibited'

        queue = deque([u])
        visited = set()

        while queue:
            curr = queue.popleft()
            if curr in visited or activation_state[curr] == 'inhibited':
                continue
            visited.add(curr)

            # Activate current node
            activation_state[curr] = 'active'

            if '-' in curr:
                #  check orthogonality and descend
                if np.dot(u_code, self.CombForest[curr]['code']) != 0:
                    def get_connected(node):
                        if '-' not in node:
                            if np.dot(self.CombForest[node]['code'], self.CombForest[curr]['code']) != 0:
                                return [node]
                            else:
                                return []
                        else:
                            connected = []
                            for child in self.CombForest[node]['children']:
                                connected += get_connected(child)
                            return connected
                    connected = get_connected(curr)
                    for other in connected:
                        if other != u:
                            neighbors[other] = self.CombForest[other]  #  directly store dict

            # Propagate only if not inhibited
            for parent_key in ['parentleft', 'parentright']:
                parent = self.CombForest[curr].get(parent_key)
                if (parent and parent not in visited and
                    activation_state[parent] != 'inhibited'):
                    queue.append(parent)

        return neighbors, dict(activation_state)


    # : Mutual Neighbors with Cross-Activation
    def find_mutual_neighbors_cross_activation(self, u, v, sync_threshold=1):
        """
        Find mutual neighbors by having two activation processes (from u and v)
        that must synchronize at edge nodes to reveal mutual connections.
        """
        # Run dual activation processes
        activation_u = defaultdict(float)
        activation_v = defaultdict(float)
        neighbors_u = defaultdict(float)
        neighbors_v = defaultdict(float)
        mutual_neighbors = set()
        u_code = self.CombForest[u]['code']
        v_code = self.CombForest[v]['code']

        # Initialize
        activation_u[u] = 1.0
        activation_v[v] = 1.0

        # Simultaneous BFS from both nodes
        queue_u = deque([(u, 1.0)])
        queue_v = deque([(v, 1.0)])
        visited_u = set()
        visited_v = set()

        max_steps = 10
        step = 0

        while (queue_u or queue_v) and step < max_steps:
            step += 1

            # Process u's activation
            if queue_u:
                curr_u, energy_u = queue_u.popleft()
                if curr_u not in visited_u and energy_u > 0.1:
                    visited_u.add(curr_u)
                    activation_u[curr_u] = energy_u

                    if '-' in curr_u:
                        curr_code = self.CombForest[curr_u]['code']
                        if np.dot(u_code, curr_code) != 0:
                            connected = self.get_connected(curr_u, curr_code)
                            for other in connected:
                                if other != u:
                                    neighbors_u[other] = max(neighbors_u[other], energy_u)

                    for parent_key in ['parentleft', 'parentright']:
                        parent = self.CombForest[curr_u].get(parent_key)
                        if parent and parent not in visited_u:
                            queue_u.append((parent, energy_u * 0.8))

            # Process v's activation
            if queue_v:
                curr_v, energy_v = queue_v.popleft()
                if curr_v not in visited_v and energy_v > 0.1:
                    visited_v.add(curr_v)
                    activation_v[curr_v] = energy_v

                    if '-' in curr_v:
                        curr_code = self.CombForest[curr_v]['code']
                        if np.dot(v_code, curr_code) != 0:
                            connected = self.get_connected(curr_v, curr_code)
                            for other in connected:
                                if other != v:
                                    neighbors_v[other] = max(neighbors_v[other], energy_v)

                    for parent_key in ['parentleft', 'parentright']:
                        parent = self.CombForest[curr_v].get(parent_key)
                        if parent and parent not in visited_v:
                            queue_v.append((parent, energy_v * 0.8))

        # Find mutual neighbors where strengths synchronize
        for w in set(neighbors_u.keys()) & set(neighbors_v.keys()):
            if w != u and w != v:
                combined_strength = neighbors_u[w] + neighbors_v[w]
                if combined_strength > sync_threshold:
                    mutual_neighbors.add(w)

        return mutual_neighbors, (activation_u, activation_v)


# Example usage functions
def demonstrate_activation_methods(CombForest, test_node):
    """Demonstrate all activation methods"""
    finder = ActivationNeighborFinder(CombForest)

    print(f"\n=== Activation Methods for Node '{test_node}' ===")

    #  Inhibition activation
    neighbors5, states = finder.find_neighbors_inhibition_activation(test_node)
    print(f" Inhibition Activation: {neighbors5}")

    # Compare with original method
    original_neighbors = find_neighbors_original(test_node, CombForest)
    print(f"Original Method: {original_neighbors}")

    return {
        'inhibition': neighbors5,
        'original': original_neighbors
    }

def find_neighbors_original(u, CombForest):
    """Original neighbor finding method for comparison"""
    neighbors = set()
    from collections import deque
    queue = deque([u])
    visited = set()
    while queue:
        curr = queue.popleft()
        if curr in visited:
            continue
        visited.add(curr)
        pl = CombForest[curr]['parentleft']
        if pl is not None and pl not in visited:
            queue.append(pl)
        pr = CombForest[curr]['parentright']
        if pr is not None and pr not in visited:
            queue.append(pr)
        if '-' in curr:
            x, y = curr.split('-')
            if u == x:
                neighbors.add(y)
            elif u == y:
                neighbors.add(x)
    return neighbors
Results=demonstrate_activation_methods(CombForest, 'A')


Resul=demonstrate_activation_methods(CombForest, 'B')
print("-------- Mutual--------------")
finder = ActivationNeighborFinder(CombForest)
Mn=finder.find_mutual_neighbors_cross_activation('A', 'B')
print(Mn)


=== Activation Methods for Node 'A' ===
 Inhibition Activation: {'D': {'parentleft': 'A-D', 'parentright': None, 'children': [], 'code': array([1., 0., 0., 0., 1., 1., 0., 0., 0., 1.])}, 'C': {'parentleft': 'A-C', 'parentright': None, 'children': [], 'code': array([0., 1., 0., 0., 0., 1., 1., 0., 1., 0.])}, 'E': {'parentleft': 'A-E', 'parentright': None, 'children': [], 'code': array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0.])}, 'B': {'parentleft': 'A-B', 'parentright': None, 'children': [], 'code': array([0., 0., 0., 1., 1., 0., 0., 1., 1., 0.])}}
Original Method: {'D', 'C', 'B', 'E'}

=== Activation Methods for Node 'B' ===
 Inhibition Activation: {'A': {'parentleft': 'A-D', 'parentright': None, 'children': [], 'code': array([0., 0., 1., 0., 0., 0., 1., 1., 0., 1.])}, 'C': {'parentleft': 'A-C', 'parentright': None, 'children': [], 'code': array([0., 1., 0., 0., 0., 1., 1., 0., 1., 0.])}, 'E': {'parentleft': 'A-E', 'parentright': None, 'children': [], 'code': array([1., 1., 1., 1., 0.,

# Testing orthogonality
Testing nodes orthogonality by serching for neighbors we are testing each time if codes are not orthogonal then edge node is connected to neighbor else if it is the not neighbor

In [ ]:
from collections import deque
import numpy as np

def find_neighbors(u, CombForest):
    neighbors = set()
    queue = deque([u])
    visited = set()
    while queue:
        curr = queue.popleft()
        if curr in visited:
            continue
        visited.add(curr)
        pl = CombForest[curr]['parentleft']
        if pl is not None and pl not in visited:
            queue.append(pl)
        pr = CombForest[curr]['parentright']
        if pr is not None and pr not in visited:
            queue.append(pr)
        if '-' in curr:
            x, y = curr.split('-')
            if u == x:
                neighbors.add(y)
            elif u == y:
                neighbors.add(x)
    return neighbors

def find_neighbors_new(u, CombForest):
    neighbors = set()
    queue = deque([u])
    visited = set()
    while queue:
        curr = queue.popleft()
        if curr in visited:
            continue
        visited.add(curr)
        pl = CombForest[curr]['parentleft']
        if pl is not None and pl not in visited:
            queue.append(pl)
        pr = CombForest[curr]['parentright']
        if pr is not None and pr not in visited:
            queue.append(pr)
        if '-' in curr:
            # Check if codes are not orthogonal (dot product != 0)
            if np.dot(CombForest[u]['code'], CombForest[curr]['code']) != 0:
                # Find the other original node connected to this edge
                original_nodes = [k for k in CombForest if '-' not in k]
                for other in original_nodes:
                    if other != u and np.dot(CombForest[other]['code'], CombForest[curr]['code']) != 0:
                        neighbors.add(other)
                        break  # Only one other endpoint
    return neighbors

# Example usage to check if same neighbors are found
# Assuming CombForest is built, and choosing node 'A' (adjust if needed)
n_original = find_neighbors('F', CombForest)
n_new = find_neighbors_new('F', CombForest)
print("Original neighbors:", n_original)
print("New neighbors:", n_new)
print("Same neighbors:", n_original == n_new)

Original neighbors: {'S', 'R', 'K', 'P'}
New neighbors: {'S', 'R', 'K', 'P'}
Same neighbors: True


In [ ]:
from collections import deque
import numpy as np

def find_neighbors(u, CombForest):
    neighbors = {}
    queue = deque([u])
    visited = set()
    while queue:
        curr = queue.popleft()
        if curr in visited:
            continue
        visited.add(curr)
        pl = CombForest[curr]['parentleft']
        if pl is not None and pl not in visited:
            queue.append(pl)
        pr = CombForest[curr]['parentright']
        if pr is not None and pr not in visited:
            queue.append(pr)
        if '-' in curr:
            x, y = curr.split('-')
            if u == x:
                neighbors[y] = CombForest[y]
            elif u == y:
                neighbors[x] = CombForest[x]
    return neighbors

def find_neighbors_new(u, CombForest):
    neighbors = {}
    queue = deque([u])
    visited = set()
    while queue:
        curr = queue.popleft()
        if curr in visited:
            continue
        visited.add(curr)
        pl = CombForest[curr]['parentleft']
        if pl is not None and pl not in visited:
            queue.append(pl)
        pr = CombForest[curr]['parentright']
        if pr is not None and pr not in visited:
            queue.append(pr)
        if '-' in curr:
            # Check if codes are not orthogonal (dot product != 0)
            if np.dot(CombForest[u]['code'], CombForest[curr]['code']) != 0:
                # Descend in the comb tree to find the other original node
                def get_connected(node):
                    if '-' not in node:
                        if np.dot(CombForest[node]['code'], CombForest[curr]['code']) != 0:
                            return [node]
                        else:
                            return []
                    else:
                        connected = []
                        for child in CombForest[node]['children']:
                            connected += get_connected(child)
                        return connected
                connected = get_connected(curr)
                for other in connected:
                    if other != u:
                        neighbors[other] = CombForest[other]
    return neighbors

# Example usage
# Assuming CombForest is defined
n_original = find_neighbors('F', CombForest)
n_new = find_neighbors_new('F', CombForest)
print("Original neighbors:", n_original)
print("New neighbors:", n_new)
print("Same neighbors:", n_original == n_new)

Original neighbors: {'K': {'parentleft': 'B-K', 'parentright': None, 'children': [], 'code': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 1., 0.])}, 'S': {'parentleft': 'C-S', 'parentright': None, 'children': [], 'code': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])}, 'P': {'parentleft': 'A-P', 'parentright': None, 'children': [], 'code': array([0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])}, 'R': {'parentleft': 'A-R', 'parentright': None, 'children': [], 'code': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0